# 1 问题引入

在讨论文本分类问题之前，先讨论一种普遍的分类问题，如构造一个垃圾邮件分类系统，它能把邮件分类为垃圾邮件和正常的邮件。这就是分类问题(Classification Problem)：给定多个数据点（在这种情况下，每一篇文章都是一个数据点），目标是把这些文章分为两类或更多类(class)。在分类问题中，给定一个训练集和相应的类别标签(label)（通常都是一些离散值），我们需要用这些训练数据（如邮件及其对应的两个标签之一）去学习出某种关系，利用这个关系我们可以去预测新的未知的数据的标签。下面，我们将详细讨论一种简单却有效的分类问题求解模型：朴素贝叶斯分类器(Naive Bayes Classifier)。

# 2 计算模型

## 2.1 朴素贝叶斯
训练一个文本分类模型，我们需要一些已经一些预先分类好的数据。但是为了学习到有效的东西，我们需要从文本中抽取出一些特定的属性，即特征(feature)。特征可以是任何东西，如单词数量、文本模式等等。这些抽取出来的特征往往要取决于解决的问题，选择的特征对模型的性能有很大的影响。

现在假设你有一个含有$n$个单词的词表(dictionary)，从每篇文章中，你抽取了一个特征向量(feature vector) $F\in \mathbf{R}^n$，$F$中的第$i$个随机变量$F_i$代表第$i$个词表中的单词的特征值，可以是出现频率(frequencey)，IF，IDF等等。使用以上定义，我们可以用数学语言去描述出如何预测某篇文章属于哪个分类：如果我们能知道每个特征$F_i$与标签$Y$之间的联合概率分布，我们就能在已知特征向量的情况下计算出该文章属于某种分类$C_i$的概率值，即:
$$
P(Y=C_i|F_1=f_1,\cdots,F_n=f_n)
$$
我们只需要计算出所有文章属于$C_i$的概率，取最高概率的$C_i$作为文章的分类标签即可。但是，问题在于我们很难估算出每个特征$F_i$与标签$Y$之间的联合概率分布，即使估算出，分布表的条目将会非常多，在实际应用中几乎无法实现。因此，我们需要简化模型，做出如下假设：给定一个类别标签，每个特征$F_i$都独立于其他的特征。这是一个强模型假设(这就是朴素的原因)，但是它大大简化模型复杂度且效果较好。上述假设可以用如下的贝叶斯网来描述：

使用简化过的模型，我们可以根据贝叶斯网做出预测，给定观察值$F_1,\cdots,F_n$，选择$Y$的某个值使得条件概率分布值最大：
\begin{equation}
\begin{aligned}
pred(f_1,\cdots,f_n)
=& argmax_C P(Y=C|F_1=f_1,\cdots,F_N=f_n)\\
=& argmax_C P(Y=C,F_1=f_1,\cdots,F_N=f_n)\\
=& argmax_C P(Y=C) \prod_{i=1}^n P(F_i=f_i|Y=C)\\
\end{aligned}
\end{equation}
把问题变得更为一般化：假设现在有$k$个分类标签，我们可以计算出:
$$
P(Y,F_1=f_1,\cdots,F_n=f_n)=
\begin{bmatrix}
P(Y=C_1) \prod_{i}P(F_i=f_i|Y=C_1))\\
P(Y=C_2) \prod_{i}P(F_i=f_i|Y=C_2))\\
\vdots\\
P(Y=C_k) \prod_{i}P(F_i=f_i|Y=C_k))
\end{bmatrix}
$$
根据特征向量$F$预测得到的标签就是：
$$
pred(F)=argmax_{C_i}P(Y=C_i)\prod_j P(F_j=f_j|Y=C_i)
$$
模型建立好之后，我们需要从训练集中得到某个类别标签的概率分布$P(Y=C)$和先验概率$P(F|Y=C)$，这就是下面要讨论的问题：参数估计(parameter estimation)。

## 2.2 参数估计
有$N$个样本点或观察值$x_1,\cdots,x_n$，假设这些数据来源于参数为未知量$\theta$的分布，即出现$x_i$的概率为$P_{\theta}(x_i)$。

如何从样本中学习出最有可能的$\theta$值？最常用的方法是最大似然估计(MLE)。它要求样本服从独立同分布。定义以下的似然函数：
$$
\mathcal{L}(\theta)=P_{\theta}(x_1,\cdots,x_N)
$$
由于样本服从独立同分布：
$$
\mathcal{L}(\theta)=\prod_{i=1}^N P_{\theta}(x_i)
$$
我们需要找到某个$\theta$使得似然函数的值最大，采用以下计算方法：
$$
\frac{\partial}{\partial \theta}\mathcal{L}(\theta)=0
$$

## 2.3 朴素贝叶斯的最大似然估计
现在我们来估计朴素贝叶斯中的未知参量，先定义如下变量：
1. $n$-词典里的单词数。
2. $N$-训练集数目。$N_C$为训练集中类别为$C$的样本。
3. $F_i$-词典中第$i$个单词的特征值，为了简化推导，假设$F_i$只能取0或1。
4. $Y$-类别随机变量。
5. $f_i^{(j)}$-第$j$个样本点的第$i$个特征。

设$\theta=P(F_i=1|Y=C)$，则：
$$
\mathcal{L}(\theta)=\prod_{j=1}^{N_C} P(F_i=f_i^{(j)}|Y=C)=\prod_{j=1}^{N_C} \theta^{f_i^{(j)}}(1-\theta)^{1-f_i^{(j)}}
$$
$$
\log\mathcal{L}(\theta)=\log(\theta)\sum_{j=1}^{N_C}f_i^{(j)}+\log(1-\theta)\sum_{j=1}^{N_C}(1-f_i^{(j)})
$$
$$
\frac{\partial}{\partial \theta}\log\mathcal{L}(\theta)=0
$$
$$
\theta=\frac{1}{N_C}\sum_{j=1}^{N_C}f_i^{(j)}
$$

## 2.4 平滑
最大似然估计虽然很有效，但是如果出现估计出的概率为0的情况，将会极大地影响模型的性能。因此，我们需要对分类器做平滑处理(smooth)，最常用的是拉普拉斯平滑：
$$
P_{MLE}(x)=\frac{count(x)}{N}
$$
$$
P_{LAP,k}(x)=\frac{count(x+k)}{N+k|X|}
$$

# 3 编程实现

# 4 模型评估

In [9]:
import scipy.io
import random
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import importlib
import sys 
import os
sys.path.append(".")
# reload module in case that module changes
import text_preprocessing
importlib.reload(text_preprocessing)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jsjhf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\jsjhf\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jsjhf\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


<module 'text_preprocessing' from 'C:\\Users\\jsjhf\\Desktop\\course\\coursework\\NLP\\hw\\hw2\\text_preprocessing.py'>

In [10]:
# load txt files
base_path = "20_newsgroups"
class_paths = os.listdir(base_path)
txt_paths = []
txt_labels = []
for i in range(len(class_paths)):
    files = os.listdir(os.path.join(base_path, class_paths[i]))
    for file in files:
        txt_paths.append(os.path.join(base_path, os.path.join(class_paths[i], file)))
        txt_labels.append(i)

In [11]:
# text preprocessing
data = text_preprocessing.text_preprocessing(txt_paths)

Start preprocessing...
current: 2000
current: 4000
current: 6000
current: 8000
current: 10000
current: 12000
current: 14000
current: 16000
current: 18000
Finish preprocessing !


In [12]:
# save preprocessed data
news_data = {"data": data, "labels": np.array(txt_labels).reshape((len(txt_labels), 1))}
scipy.io.savemat('data/news_data.mat', news_data)

In [13]:
# load data
load_data = scipy.io.loadmat('data/news_data.mat')
news_data = load_data['data']
news_labels = load_data['labels']

# shuffle
zipped_data = list(zip(news_data, news_labels))  
random.seed(0)
random.shuffle(zipped_data)
new_zipped_data = list(map(list, zip(*zipped_data)))  
news_data, news_labels = np.array(new_zipped_data[0]), np.array(new_zipped_data[1])  

# split data into training, validation and test sets
training_data = news_data[:15000, :]
training_labels = news_labels[:15000]
validation_data = news_data[15000:17500, :]
validation_labels = news_labels[15000:17500]
test_data = news_data[17500:, :]
test_labels = news_labels[17500:]

In [14]:
# Naive Bayes Classifier
class NBC:
    def __init__(self):
        # P(y=c)
        self.log_Pc = 0
        
        # P(F=f|Y=c)
        self.log_prior = 0
        
        # training size
        self.N = 0
        
        # dimension
        self.d = 0
        self.class_size = 0
        
    def compute_log_Pc(self, data, labels):
        log_training_size = np.log(self.N)
        for c in range(self.class_size):
            c_data = data[labels.ravel()==c, :]
            self.log_Pc[c] = np.log(c_data.shape[0]) - log_training_size
            
    def compute_log_prior(self, data, labels, k):
        for c in range(self.class_size):
            c_data = data[labels.ravel()==c, :]
            all_cnt = c_data.sum()
            for f in range(self.d):
                log_cnt = np.log(c_data[:, f].sum()+k)
                log_all_cnt = np.log(all_cnt+k*d)
                self.log_prior[c][f] = log_cnt - log_all_cnt
    
    def fit(self, data, labels, k):
        print("Start fitting...")
        self.N = data.shape[0]
        self.d = data.shape[1]
        self.class_size = len(set(labels.ravel()))
        self.log_Pc = np.zeros((self.class_size, ))
        self.log_prior = np.zeros((self.class_size, self.d))
        self.compute_log_Pc(data, labels)
        self.compute_log_prior(training_data, labels, k)
        print("Finish fitting !")
    
    # Input: n×d
    # Output: n×1
    def predict(self, data):
        pred_list = []
        for X in data:
            max_log_prob = float('-inf')
            pred_c = 0
            for c in range(self.class_size):
                log_prob = self.log_Pc[c]
                for f in range(self.d):
                    log_prob += self.log_prior[c][f]*np.log(1+X[f])
                if log_prob > max_log_prob:
                    max_log_prob = log_prob
                    pred_c = c
            pred_list.append(pred_c)
        return pred_list
    
    def accuracy(self, data, labels):
        print("Start computing accuracy...")
        pred_list = self.predict(data)
        n = len(labels)
        n_accuracy = (pred_list == labels.ravel()).sum()
        print("Finish computing accuracy !")
        return n_accuracy/n

In [15]:
nbc = NBC()
nbc.fit(training_data, training_labels, 1)
nbc.accuracy(validation_data[0:1000, :], validation_labels[0:1000, :])

Start fitting...
Finish fitting !
Start computing accuracy...
Finish computing accuracy !


0.837